In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os

folder_path = '/content/drive/My Drive/Team2/ML project input data/'

files = os.listdir(folder_path)

df = {}

for file in files:
    if file.endswith(".csv"):
      print(file)
      file_path = os.path.join(folder_path, file)
      # Read the CSV file and store it in a dictionary with the filename as the key
      df[file[:-4]] = pd.read_csv(file_path)  # file[:-4] to remove '.csv' from the filename

return.csv
pe.csv


<ipython-input-4-bb8970ba0dea>:15: DtypeWarning: Columns (7,66) have mixed types. Specify dtype option on import or set low_memory=False.
  df[file[:-4]] = pd.read_csv(file_path)  # file[:-4] to remove '.csv' from the filename


volume.csv
volatility.csv


In [ ]:
df_return = df['return']
df_pe = df['pe']
df_volatility = df['volatility']
df_volume = df['volume']

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import numpy as np
import warnings
from sklearn.metrics import mean_squared_error

warnings.filterwarnings('ignore')

In [ ]:
def create_sequences(data, features, target, lookback):
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[features].iloc[i-lookback:i].values)
        y.append(data[target].iloc[i])
    return np.array(X), np.array(y)

In [ ]:
features = ['volatility', 'P/E', 'volume', 'return']
target = 'return'

In [ ]:
performance = {}
outcome = []

**Training Batch 1: Company 1 - 10**

In [ ]:
for c in ["VTM.JST.CMLBK.FOR INTRD.", "VIETNAM INTERNATIONAL COMMERCIAL", "THANH THANH CONG - BIEN HOA", "PETROVIETNAM FCM.", "HA DO GROUP", "PHUNHUAN JEWELRY", "ELCOM TECHNOLOGY COMMUNICATIONS", "FECON MINING", "DONG PHU RUBBER", "CNG VIETNAM"]:
  # Get a specific stock's data
  df_comp = df_return[['Date', c]]
  df_comp.rename(columns = {c: "return"}, inplace=True)
  df_comp['date'] = pd.to_datetime(df_comp['Date'])
  # merge data to get volatility, P/E ratio, and volume
  df_comp = pd.merge(df_comp, df_volatility[['Date', c]], on='Date')
  df_comp = pd.merge(df_comp, df_pe[['Date', c]], on='Date')
  df_comp = pd.merge(df_comp, df_volume[['Date', c]], on='Date')
  df_comp.rename(columns = {c+'_x': 'volatility', c+'_y': 'P/E', c: 'volume'}, inplace=True)
  df_comp = df_comp.drop(columns = ['Date'])
  df_comp = df_comp.dropna()

  features = ['volatility', 'P/E', 'volume', 'return']
  target = 'return'

  # Scale for each columns
  df_comp_scaled = pd.DataFrame()
  for f in features:
    df_comp[f+"_mean"] = df_comp[f].rolling(window=60, min_periods=60).mean()
    df_comp[f+"_std"] = df_comp[f].rolling(window=60, min_periods=60).std()
    df_comp_scaled[f] = (df_comp[f] - df_comp[f+"_mean"])/df_comp[f+"_std"]
  df_comp_scaled = df_comp_scaled.reset_index()
  df_comp_scaled = df_comp_scaled.dropna()

  lookback = 60  # Can test different number of days to look back to
  X, y = create_sequences(df_comp_scaled, features, target, lookback)

  # Train - Test split
  train_size =  len(X) - len(df_comp[df_comp["date"] >= "2022-01-01"])
  X_train, X_test = X[:train_size], X[train_size:]
  y_train, y_test = y[:train_size], y[train_size:]

  model = Sequential()
  model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
  model.add(Dropout(0.2))
  model.add(LSTM(units=50, return_sequences=False))
  model.add(Dropout(0.2))
  model.add(Dense(units=1))

  model.compile(optimizer='adam', loss='mean_squared_error')
  lstm = model.fit(X_train, y_train, epochs=30, batch_size= 32, validation_split=0.2, verbose=0, shuffle=False)

  trainPredict = model.predict(X_train)
  testPredict = model.predict(X_test)

  trainPredict1 = np.array([i[0] for i in trainPredict])*np.array(df_comp["return_std"][-len(X):-len(testPredict)]) + df_comp["return_mean"][-len(X):-len(testPredict)]
  trainY = (y_train*df_comp["return_std"][-len(X):-len(testPredict)]) + df_comp["return_mean"][-len(X):-len(testPredict)]
  testPredict1 = np.array([i[0] for i in testPredict])*np.array(df_comp["return_std"][-len(testPredict):]) + df_comp["return_mean"][-len(testPredict):]
  testY = (y_test*df_comp["return_std"][-len(testPredict):]) + df_comp["return_mean"][-len(testPredict):]


  trainScore = np.sqrt(mean_squared_error(trainY, trainPredict1))
  testScore = np.sqrt(mean_squared_error(testY, testPredict1))

  performance[c] = [trainScore, testScore]

  l = [df_comp.date[-len(y_test):], testY, testPredict1]
  pred_comp = pd.DataFrame()
  pred_comp["date"] = l[0]
  pred_comp["actual_return"] = l[1]
  pred_comp["predicted_return"] = l[2]
  outcome.append(pred_comp)

  print("Done with "+c)

19/19 [==============================] - 0s 23ms/step
Done with VTM.JST.CMLBK.FOR INTRD.
19/19 [==============================] - 1s 15ms/step
Done with VIETNAM INTERNATIONAL COMMERCIAL
19/19 [==============================] - 0s 22ms/step
Done with THANH THANH CONG - BIEN HOA
19/19 [==============================] - 0s 24ms/step
Done with PETROVIETNAM FCM.
19/19 [==============================] - 0s 17ms/step
Done with HA DO GROUP
19/19 [==============================] - 0s 15ms/step
Done with PHUNHUAN JEWELRY
19/19 [==============================] - 0s 20ms/step
Done with ELCOM TECHNOLOGY COMMUNICATIONS
19/19 [==============================] - 0s 15ms/step
Done with FECON MINING
19/19 [==============================] - 0s 16ms/step
Done with DONG PHU RUBBER
19/19 [==============================] - 0s 22ms/step
Done with CNG VIETNAM


In [ ]:
for i, c in enumerate(["VTM.JST.CMLBK.FOR INTRD.", "VIETNAM INTERNATIONAL COMMERCIAL", "THANH THANH CONG - BIEN HOA", "PETROVIETNAM FCM.",
          "HA DO GROUP", "PHUNHUAN JEWELRY", "ELCOM TECHNOLOGY COMMUNICATIONS", "FECON MINING", "DONG PHU RUBBER", "CNG VIETNAM"]):
  outcome[i].to_csv(f"/content/drive/MyDrive/Team2/ML project output data/{c}.csv")



**Training Batch 2: Company 11 - 20**

In [ ]:
for c in ["GELEX GROUP", "MOBILE WORLD INVESTMENT", "AN PHAT PLAST.&GRN.ENVM.", "VIETNAM DAIRY PRODUCTS", "DUCGIANG CHEMICALS GROUP",
                       "DANANG RUBBER","FIRST REAL JOINT STOCK COMPANY","VINH SON-SONG HINH HYPW.","PETROLIMEX GAS", "INDUSTRIAL URBAN DEV."]:
  # Get a specific stock's data
  df_comp = df_return[['Date', c]]
  df_comp.rename(columns = {c: "return"}, inplace=True)
  df_comp['date'] = pd.to_datetime(df_comp['Date'])
  # merge data to get volatility, P/E ratio, and volume
  df_comp = pd.merge(df_comp, df_volatility[['Date', c]], on='Date')
  df_comp = pd.merge(df_comp, df_pe[['Date', c]], on='Date')
  df_comp = pd.merge(df_comp, df_volume[['Date', c]], on='Date')
  df_comp.rename(columns = {c+'_x': 'volatility', c+'_y': 'P/E', c: 'volume'}, inplace=True)
  df_comp = df_comp.drop(columns = ['Date'])
  df_comp = df_comp.dropna()

  features = ['volatility', 'P/E', 'volume', 'return']
  target = 'return'

  # Scale for each columns
  df_comp_scaled = pd.DataFrame()
  for f in features:
    df_comp[f+"_mean"] = df_comp[f].rolling(window=60, min_periods=60).mean()
    df_comp[f+"_std"] = df_comp[f].rolling(window=60, min_periods=60).std()
    df_comp_scaled[f] = (df_comp[f] - df_comp[f+"_mean"])/df_comp[f+"_std"]
  df_comp_scaled = df_comp_scaled.reset_index()
  df_comp_scaled = df_comp_scaled.dropna()

  lookback = 60  # Can test different number of days to look back to
  X, y = create_sequences(df_comp_scaled, features, target, lookback)

  # Train - Test split
  train_size =  len(X) - len(df_comp[df_comp["date"] >= "2022-01-01"])
  X_train, X_test = X[:train_size], X[train_size:]
  y_train, y_test = y[:train_size], y[train_size:]

  model = Sequential()
  model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
  model.add(Dropout(0.2))
  model.add(LSTM(units=50, return_sequences=False))
  model.add(Dropout(0.2))
  model.add(Dense(units=1))

  model.compile(optimizer='adam', loss='mean_squared_error')
  lstm = model.fit(X_train, y_train, epochs=30, batch_size= 32, validation_split=0.2, verbose=0, shuffle=False)

  trainPredict = model.predict(X_train)
  testPredict = model.predict(X_test)

  trainPredict1 = np.array([i[0] for i in trainPredict])*np.array(df_comp["return_std"][-len(X):-len(testPredict)]) + df_comp["return_mean"][-len(X):-len(testPredict)]
  trainY = (y_train*df_comp["return_std"][-len(X):-len(testPredict)]) + df_comp["return_mean"][-len(X):-len(testPredict)]
  testPredict1 = np.array([i[0] for i in testPredict])*np.array(df_comp["return_std"][-len(testPredict):]) + df_comp["return_mean"][-len(testPredict):]
  testY = (y_test*df_comp["return_std"][-len(testPredict):]) + df_comp["return_mean"][-len(testPredict):]


  trainScore = np.sqrt(mean_squared_error(trainY, trainPredict1))
  testScore = np.sqrt(mean_squared_error(testY, testPredict1))

  performance[c] = [trainScore, testScore]

  l = [df_comp.date[-len(y_test):], testY, testPredict1]
  pred_comp = pd.DataFrame()
  pred_comp["date"] = l[0]
  pred_comp["actual_return"] = l[1]
  pred_comp["predicted_return"] = l[2]
  outcome.append(pred_comp)

  print("Done with "+c)


19/19 [==============================] - 0s 16ms/step
Done with GELEX GROUP
19/19 [==============================] - 1s 17ms/step
Done with MOBILE WORLD INVESTMENT
19/19 [==============================] - 0s 18ms/step
Done with AN PHAT PLAST.&GRN.ENVM.
19/19 [==============================] - 0s 17ms/step
Done with VIETNAM DAIRY PRODUCTS
19/19 [==============================] - 0s 16ms/step
Done with DUCGIANG CHEMICALS GROUP
19/19 [==============================] - 0s 21ms/step
Done with DANANG RUBBER
19/19 [==============================] - 0s 17ms/step
Done with FIRST REAL JOINT STOCK COMPANY
19/19 [==============================] - 0s 18ms/step
Done with VINH SON-SONG HINH HYPW.
19/19 [==============================] - 0s 17ms/step
Done with PETROLIMEX GAS
19/19 [==============================] - 0s 17ms/step
Done with INDUSTRIAL URBAN DEV.


In [ ]:
for i, c in enumerate(["GELEX GROUP", "MOBILE WORLD INVESTMENT", "AN PHAT PLAST.&GRN.ENVM.", "VIETNAM DAIRY PRODUCTS", "DUCGIANG CHEMICALS GROUP",
                       "DANANG RUBBER","FIRST REAL JOINT STOCK COMPANY","VINH SON-SONG HINH HYPW.","PETROLIMEX GAS", "INDUSTRIAL URBAN DEV."]):
  outcome[i].to_csv(f"/content/drive/MyDrive/Team2/ML project output data/{c}.csv")

**Training Batch 3: Company 21 - 30**

In [ ]:
for c in ["SSI SECURITIES", "PHAT DAT RLST.DEV.", "FPT", "JST.CMLBK.FOR FRT. OF VIET NAM", "NAM LONG INVESTMENT",
                       "GEMADEPT", "BAOVIET HOLDINGS", "VIET NAM TANKER", "PHU TAI", "SAO TA FOODS"]:
  # Get a specific stock's data
  df_comp = df_return[['Date', c]]
  df_comp.rename(columns = {c: "return"}, inplace=True)
  df_comp['date'] = pd.to_datetime(df_comp['Date'])
  # merge data to get volatility, P/E ratio, and volume
  df_comp = pd.merge(df_comp, df_volatility[['Date', c]], on='Date')
  df_comp = pd.merge(df_comp, df_pe[['Date', c]], on='Date')
  df_comp = pd.merge(df_comp, df_volume[['Date', c]], on='Date')
  df_comp.rename(columns = {c+'_x': 'volatility', c+'_y': 'P/E', c: 'volume'}, inplace=True)
  df_comp = df_comp.drop(columns = ['Date'])
  df_comp = df_comp.dropna()

  features = ['volatility', 'P/E', 'volume', 'return']
  target = 'return'

  # Scale for each columns
  df_comp_scaled = pd.DataFrame()
  for f in features:
    df_comp[f+"_mean"] = df_comp[f].rolling(window=60, min_periods=60).mean()
    df_comp[f+"_std"] = df_comp[f].rolling(window=60, min_periods=60).std()
    df_comp_scaled[f] = (df_comp[f] - df_comp[f+"_mean"])/df_comp[f+"_std"]
  df_comp_scaled = df_comp_scaled.reset_index()
  df_comp_scaled = df_comp_scaled.dropna()

  lookback = 60  # Can test different number of days to look back to
  X, y = create_sequences(df_comp_scaled, features, target, lookback)

  # Train - Test split
  train_size =  len(X) - len(df_comp[df_comp["date"] >= "2022-01-01"])
  X_train, X_test = X[:train_size], X[train_size:]
  y_train, y_test = y[:train_size], y[train_size:]

  model = Sequential()
  model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
  model.add(Dropout(0.2))
  model.add(LSTM(units=50, return_sequences=False))
  model.add(Dropout(0.2))
  model.add(Dense(units=1))

  model.compile(optimizer='adam', loss='mean_squared_error')
  lstm = model.fit(X_train, y_train, epochs=30, batch_size= 32, validation_split=0.2, verbose=0, shuffle=False)

  trainPredict = model.predict(X_train)
  testPredict = model.predict(X_test)

  trainPredict1 = np.array([i[0] for i in trainPredict])*np.array(df_comp["return_std"][-len(X):-len(testPredict)]) + df_comp["return_mean"][-len(X):-len(testPredict)]
  trainY = (y_train*df_comp["return_std"][-len(X):-len(testPredict)]) + df_comp["return_mean"][-len(X):-len(testPredict)]
  testPredict1 = np.array([i[0] for i in testPredict])*np.array(df_comp["return_std"][-len(testPredict):]) + df_comp["return_mean"][-len(testPredict):]
  testY = (y_test*df_comp["return_std"][-len(testPredict):]) + df_comp["return_mean"][-len(testPredict):]


  trainScore = np.sqrt(mean_squared_error(trainY, trainPredict1))
  testScore = np.sqrt(mean_squared_error(testY, testPredict1))

  performance[c] = [trainScore, testScore]

  l = [df_comp.date[-len(y_test):], testY, testPredict1]
  pred_comp = pd.DataFrame()
  pred_comp["date"] = l[0]
  pred_comp["actual_return"] = l[1]
  pred_comp["predicted_return"] = l[2]
  outcome.append(pred_comp)

  print("Done with "+c)


19/19 [==============================] - 1s 17ms/step
Done with SSI SECURITIES
19/19 [==============================] - 0s 16ms/step
Done with PHAT DAT RLST.DEV.
19/19 [==============================] - 0s 17ms/step
Done with FPT
19/19 [==============================] - 0s 25ms/step
Done with JST.CMLBK.FOR FRT. OF VIET NAM
19/19 [==============================] - 1s 28ms/step
Done with NAM LONG INVESTMENT
19/19 [==============================] - 0s 16ms/step
Done with GEMADEPT
19/19 [==============================] - 0s 17ms/step
Done with BAOVIET HOLDINGS
19/19 [==============================] - 1s 25ms/step
Done with VIET NAM TANKER
19/19 [==============================] - 1s 26ms/step
Done with PHU TAI
19/19 [==============================] - 0s 16ms/step
Done with SAO TA FOODS


In [ ]:
for i, c in enumerate(["SSI SECURITIES", "PHAT DAT RLST.DEV.", "FPT", "JST.CMLBK.FOR FRT. OF VIET NAM", "NAM LONG INVESTMENT",
                       "GEMADEPT", "BAOVIET HOLDINGS", "VIET NAM TANKER", "PHU TAI", "SAO TA FOODS"]):
  outcome[i].to_csv(f"/content/drive/MyDrive/Team2/ML project output data/{c}.csv")


**Training Batch 4: Company 31 - 40**

In [ ]:
for c in ["MILITARY COML.JST.BANK", "BANK FOR INVDL.OF VTM.", "HOCHIMINH CTY.SECS.", "BECAMEX INFR.DEVELOPMENT","DIGIWORLD",
          "BA RIA-VUNG TAU HSE.DEV.", "VINH HOAN", "CMC", "HONG HA FOOD INVESTMENT DEVELOPMENT", "BINH MINH PLASTICS"]:
  # Get a specific stock's data
  df_comp = df_return[['Date', c]]
  df_comp.rename(columns = {c: "return"}, inplace=True)
  df_comp['date'] = pd.to_datetime(df_comp['Date'])
  # merge data to get volatility, P/E ratio, and volume
  df_comp = pd.merge(df_comp, df_volatility[['Date', c]], on='Date')
  df_comp = pd.merge(df_comp, df_pe[['Date', c]], on='Date')
  df_comp = pd.merge(df_comp, df_volume[['Date', c]], on='Date')
  df_comp.rename(columns = {c+'_x': 'volatility', c+'_y': 'P/E', c: 'volume'}, inplace=True)
  df_comp = df_comp.drop(columns = ['Date'])
  df_comp = df_comp.dropna()

  features = ['volatility', 'P/E', 'volume', 'return']
  target = 'return'

  # Scale for each columns
  df_comp_scaled = pd.DataFrame()
  for f in features:
    df_comp[f+"_mean"] = df_comp[f].rolling(window=60, min_periods=60).mean()
    df_comp[f+"_std"] = df_comp[f].rolling(window=60, min_periods=60).std()
    df_comp_scaled[f] = (df_comp[f] - df_comp[f+"_mean"])/df_comp[f+"_std"]
  df_comp_scaled = df_comp_scaled.reset_index()
  df_comp_scaled = df_comp_scaled.dropna()

  lookback = 60  # Can test different number of days to look back to
  X, y = create_sequences(df_comp_scaled, features, target, lookback)

  # Train - Test split
  train_size =  len(X) - len(df_comp[df_comp["date"] >= "2022-01-01"])
  X_train, X_test = X[:train_size], X[train_size:]
  y_train, y_test = y[:train_size], y[train_size:]

  model = Sequential()
  model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
  model.add(Dropout(0.2))
  model.add(LSTM(units=50, return_sequences=False))
  model.add(Dropout(0.2))
  model.add(Dense(units=1))

  model.compile(optimizer='adam', loss='mean_squared_error')
  lstm = model.fit(X_train, y_train, epochs=30, batch_size= 32, validation_split=0.2, verbose=0, shuffle=False)

  trainPredict = model.predict(X_train)
  testPredict = model.predict(X_test)

  trainPredict1 = np.array([i[0] for i in trainPredict])*np.array(df_comp["return_std"][-len(X):-len(testPredict)]) + df_comp["return_mean"][-len(X):-len(testPredict)]
  trainY = (y_train*df_comp["return_std"][-len(X):-len(testPredict)]) + df_comp["return_mean"][-len(X):-len(testPredict)]
  testPredict1 = np.array([i[0] for i in testPredict])*np.array(df_comp["return_std"][-len(testPredict):]) + df_comp["return_mean"][-len(testPredict):]
  testY = (y_test*df_comp["return_std"][-len(testPredict):]) + df_comp["return_mean"][-len(testPredict):]


  trainScore = np.sqrt(mean_squared_error(trainY, trainPredict1))
  testScore = np.sqrt(mean_squared_error(testY, testPredict1))

  performance[c] = [trainScore, testScore]

  l = [df_comp.date[-len(y_test):], testY, testPredict1]
  pred_comp = pd.DataFrame()
  pred_comp["date"] = l[0]
  pred_comp["actual_return"] = l[1]
  pred_comp["predicted_return"] = l[2]
  outcome.append(pred_comp)

  print("Done with "+c)


19/19 [==============================] - 1s 27ms/step
Done with MILITARY COML.JST.BANK
19/19 [==============================] - 0s 17ms/step
Done with BANK FOR INVDL.OF VTM.
19/19 [==============================] - 0s 17ms/step
Done with HOCHIMINH CTY.SECS.
19/19 [==============================] - 0s 17ms/step
Done with BECAMEX INFR.DEVELOPMENT
19/19 [==============================] - 0s 23ms/step
Done with DIGIWORLD
19/19 [==============================] - 0s 18ms/step
Done with BA RIA-VUNG TAU HSE.DEV.
19/19 [==============================] - 0s 20ms/step
Done with VINH HOAN
12/12 [==============================] - 0s 17ms/step
Done with CMC
19/19 [==============================] - 0s 18ms/step
Done with HONG HA FOOD INVESTMENT DEVELOPMENT
19/19 [==============================] - 0s 18ms/step
Done with BINH MINH PLASTICS


In [ ]:
for i, c in enumerate(["MILITARY COML.JST.BANK", "BANK FOR INVDL.OF VTM.", "HOCHIMINH CTY.SECS.", "BECAMEX INFR.DEVELOPMENT","DIGIWORLD",
          "BA RIA-VUNG TAU HSE.DEV.", "VINH HOAN", "CMC", "HONG HA FOOD INVESTMENT DEVELOPMENT", "BINH MINH PLASTICS"]):
  outcome[i].to_csv(f"/content/drive/MyDrive/Team2/ML project output data/{c}.csv")

**Export the train and test MSE**

In [ ]:
pd.DataFrame(performance).to_csv(f"/content/drive/MyDrive/Team2/ML project output data/performance/performance.csv")

**Export the trading dates of the testing period for future reference**

In [ ]:
df_comp['date'][-593:].to_csv(f"/content/drive/MyDrive/Team2/ML project output data/performance/date_from_eikon.csv")